In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content')
!rm -rf mmdetection
!rm -rf 2021VRDL_HW2_test
!rm -rf 2021VRDL_HW2_train
!rm -rf 2021VRDL_HW2_test.zip
!rm -rf 2021VRDL_HW2_train.zip

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
os.chdir('/content/mmdetection')

Cloning into 'mmdetection'...
remote: Enumerating objects: 21781, done.
remote: Total 21781 (delta 0), reused 0 (delta 0), pack-reused 21781
Receiving objects: 100% (21781/21781), 25.19 MiB | 22.99 MiB/s, done.
Resolving deltas: 100% (15297/15297), done.


In [ ]:
!mkdir 2021VRDL_HW2
os.chdir('/content/mmdetection/2021VRDL_HW2')

# Google Drive
!gdown --id '17lo71EG_RgAMS8o7AdaWEzzCSvsyOGCE' --output 2021VRDL_HW2_train.zip
!gdown --id '1_xCyNxYeaByDtyihPVv0OCZ5svp2NLKp' --output 2021VRDL_HW2_test.zip

!apt-get install unzi

!unzip '2021VRDL_HW2_train.zip' -d 2021VRDL_HW2_train
!unzip '2021VRDL_HW2_test.zip' -d 2021VRDL_HW2_test

# os.chdir("/content/2021VRDL_HW1_datasets/")

In [ ]:
import os
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle

# !git clone --recursive https://github.com/cupy/cupy.git
# !cd cupy
# !pip install .

# import cupy as cp

In [ ]:
import os
os.chdir('/content/mmdetection')

In [ ]:
!pip install -r requirements/build.txt
!pip install "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip install -v -e .  # or "python setup.py develop"

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-yrlrdg4m
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-yrlrdg4m
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263925 sha256=8543d5c6fb28458df0628ff3c5ead0160ed048746d7ebae15ec7f7f1b33322cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8tgsz28/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not matc

In [ ]:
# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

     |████████████████████████████████| 2.1 MB 7.6 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 7.0.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 58.0 MB 203 kB/s 
     |████████████████████████████████| 185 kB 8.8 MB/s 


In [ ]:
# SVHN extracts data from the digitStruct.mat full numbers files.  The data can be downloaded
# the Street View House Number (SVHN)  web site: http://ufldl.stanford.edu/housenumbers.  
#
# This is an A2iA tweak (YG -9 Jan 2014) of the script found here :
# http://blog.grimwisdom.com/python/street-view-house-numbers-svhn-and-octave
#
# The digitStruct.mat files in the full numbers tars (train.tar.gz, test.tar.gz, and extra.tar.gz) 
# are only compatible with matlab.  This Python program can be run at the command line and will generate 
# a json version of the dataset.
#
# Command line usage:
#       SVHN_dataextract.py [-f input] [-o output_without_extension]
#    >  python SVHN_dataextract.py -f digitStruct.mat -o digitStruct
#
# Issues:
#    The alibility to split in several files has been removed from the original
#    script.
#

import h5py
import optparse
from json import JSONEncoder

import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

parser = optparse.OptionParser()
parser.add_option("-f", dest="fin", help="Matlab full number SVHN input file", default="digitStruct.mat")
parser.add_option("-o", dest="filePrefix", help="name for the json output file", default="digitStruct")
(options,args)= parser.parse_args()

fin = options.fin

# The DigitStructFile is just a wrapper around the h5py data.  It basically references 
#    inf:              The input h5 matlab file
#    digitStructName   The h5 ref to all the file names
#    digitStructBbox   The h5 ref to all struc data
@DATASETS.register_module()
class HW2_SVHN_Dataset(CustomDataset):

    CLASSES = ('0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0')
    # getName returns the 'name' string for for the n(th) digitStruct. 
    def getName(self,n):
        return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]]])

    # bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
    def bboxHelper(self,attr):
        if (len(attr) > 1):
            attr = [self.inf[attr[j].item()][0][0] for j in range(len(attr))]
        else:
            attr = [attr[0][0]]
        return attr

    # getBbox returns a dict of data for the n(th) bbox. 
    def getBbox(self,n):
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

    # getAllDigitStructure returns all the digitStruct from the input file.     
    def getAllDigitStructure(self):
        return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

    def load_annotations(self, ann_file):
        self.inf = h5py.File(ann_file, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']

        

        cat2label = {k: i for i, k in enumerate(self.CLASSES)}

        pictDat = self.getAllDigitStructure()
        data_infos = []

        for i in range(len(pictDat)):
            filename = f'{self.img_prefix}/{pictDat[i]["name"]}'
            # filename = '/content/mmdetection/2021VRDL_HW2/2021VRDL_HW2_train/train/' + pictDat[i]["name"]
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
            data_info = dict(filename=pictDat[i]["name"], width=width, height=height)

            # annotations processing
            bbox_names = [] # label包
            for k in range(len(pictDat[i]['height'])):    
               bbox_names.append(pictDat[i]['label'][k])
            
            bboxes = []
            for j in range(len(pictDat[i]['height'])):
               bbox = [] # 小的bbox包
               bbox.append(pictDat[i]['left'][j])
               bbox.append(pictDat[i]['top'][j])
               bbox.append(pictDat[i]['width'][j])
               bbox.append(pictDat[i]['height'][j])
               bboxes.append(bbox)         

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            # filter 'DontCare'
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if str(bbox_name) in cat2label:
                    gt_labels.append(cat2label[str(bbox_name)])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)
            
            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

    def get_ann_info(self, idx):
      return self.data_infos[idx]['ann']

In [ ]:
from mmcv import Config
# cfg = Config.fromfile('/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')
cfg = Config.fromfile('/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')


In [ ]:
lr_config = dict(policy='poly', power=0.9, min_lr=0.0001, by_epoch=False)

In [ ]:
import pathlib
print(pathlib.Path().absolute())

/content/mmdetection


In [ ]:
os.chdir('/content/mmdetection')
!rm -rf checkpoints
!mkdir checkpoints
os.chdir('/content/mmdetection')
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
!wget -c http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
      -O checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_bbox_mAP-0.398_20200504_163323-30042637.pth \
      -O checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_bbox_mAP-0.398_20200504_163323-30042637.pth

--2021-11-25 12:15:21--  https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.59
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177867103 (170M) [application/octet-stream]
Saving to: ‘checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth’

checkpoints/mask_rc 100%[===================>] 169.63M  11.6MB/s    in 19s     

2021-11-25 12:15:41 (9.14 MB/s) - ‘checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth’ saved [177867103/177867103]

--2021-11-25 12:15:41--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco

In [ ]:
os.chdir('/content/mmdetection')
from mmdet.apis import set_random_seed

classes = ('0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0')
# Modify dataset type and path
cfg.dataset_type = 'HW2_SVHN_Dataset'
cfg.data_root = '2021VRDL_HW2/'

cfg.data.test.type = 'HW2_SVHN_Dataset'
cfg.data.test.data_root = '2021VRDL_HW2/'
cfg.data.test.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.test.img_prefix = '2021VRDL_HW2_test/test/'
cfg.data.test.classes = classes

cfg.data.train.type = 'HW2_SVHN_Dataset'
cfg.data.train.data_root = '2021VRDL_HW2/'
cfg.data.train.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.train.img_prefix = '2021VRDL_HW2_train/train/'
cfg.data.train.classes = classes

cfg.data.val.type = 'HW2_SVHN_Dataset'
cfg.data.val.data_root = '2021VRDL_HW2/'
cfg.data.val.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.val.img_prefix = '2021VRDL_HW2_train/train/'
cfg.data.val.classes = classes

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 11
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_bbox_mAP-0.398_20200504_163323-30042637.pth'  # noqa


# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.models import build_detector
from mmdet.apis import train_detector

os.chdir('/content/mmdetection')
# Build dataset
# print(cfg.data.train)

datasets = [build_dataset(cfg.data.train)]


/content/mmdetection/mmdet/datasets/custom.py:157: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


In [ ]:
print(datasets[0])


HW2_SVHN_Dataset Train dataset with number of images 25933, and instance counts: 
+-----------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
| category  | count | category | count | category | count | category | count | category | count |
+-----------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
| 0 [0.0]   | 0     | 1 [1.0]  | 10676 | 2 [2.0]  | 7980  | 3 [3.0]  | 6589  | 4 [4.0]  | 5678  |
| 5 [5.0]   | 5485  | 6 [6.0]  | 4567  | 7 [7.0]  | 4355  | 8 [8.0]  | 3975  | 9 [9.0]  | 3637  |
|           |       |          |       |          |       |          |       |          |       |
| 10 [10.0] | 3932  |          |       |          |       |          |       |          |       |
+-----------+-------+----------+-------+----------+-------+----------+-------+----------+-------+


In [ ]:
os.chdir('/content/mmdetection')

In [ ]:

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
# model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

train_detector(model, datasets, cfg, distributed=False, validate=False)